# Automation to increase DevOps productivity

## Prerequisites

- Have 2 K8s clusters.
- Cluster Controller installed to the K8s clusters.
- The `cnatix` CLI is installed.
- The `kubectl` command is installed.
- The user who runs the test plan is an admin of the root org.
- `cnatix login` has already run.

To verify that a user is an admin of the root org, run:

In [ ]:
# Verify that the role binding is set to the <root-org>:admin.
cnatix iam users self

## Test case 1: Provision developer or devops engineer access to K8s API server across multiple clusters

### Expected Results

- An admin of the root org is able to create a new org.
- An admin of the root org is able to create a new user.
- An admin of the root org is able to create a new namespace for the newly created org.

### Test Execution Steps

#### *Step 1*: Create a new org

Create a new org whose parent org is the root org.

If you have already created an org in the Visibility test case, you can reuse it.

In [ ]:
# Verify the current list of orgs.
cnatix iam orgs list

In [ ]:
ROOT_ORG_UUID=<UUID of root org>
ORG_NAME=<name of the new org>
ORG_DESCRIPTION=<description of the org>

cnatix iam orgs create \
  --name "${ORG_NAME}" \
  --parent-org-uuid ${ROOT_ORG_UUID} \
  --description "${ORG_DESCRIPTION}"

# Verify that the new org is created.
cnatix iam orgs list

#### *Step 2*: Create a new user that belongs to the newly created org

In [ ]:
# Verify the current list of users.
cnatix iam users list

In [ ]:
TEST_USER_EMAIL=<email address of a new test user>
FIRST_NAME=<first name of the user>
LAST_NAME=<first name of the user>
NEW_ORG_UUID=<org UUID of the newly created org>

cnatix iam users create \
  --login ${TEST_USER_EMAIL} \
  --first-name ${FIRST_NAME} \
  --last-name ${LAST_NAME} \
  --role-binding ${NEW_ORG_UUID}:admin

# Verify that the new user is created.
cnatix iam users list

If you get a “not found” error, please wait for 5-10 minutes until the
change for the new org is propapaged.

The user will receive a sign-up email from Okta. Complete the activation and finish creating an account.

#### *Step 3*: Create a new namespace that belongs to the newly created org

In [ ]:
# Verify the current list of namespaces.
cnatix iam namespaces list

If you have skipped the test case 1, set the `ORG_NAME` shell variable now.

In [ ]:
ORG_NAME=<name of the new org>

In [ ]:
TEST_NAMESPACE=<test namespace name>

cnatix iam namespaces create \
  --name ${TEST_NAMESPACE} \
  --org "${ORG_NAME}"

# Verify that the new namespace is created.
cnatix iam namespaces list

Verify that the namespace is visible using the `kubectl` command.

In [ ]:
kubectl get namespaces

Please note that the namespace will be created in the K8s cluster only when
the org with which the namespace is associated owns the K8s cluster (=
owns at least one compute instance in the K8s cluster).

## Test case 2: Authenticate & access K8s Cluster with the `kubectl` command using the `cnatix` CLI across multiple clusters

### Expected Results

- A user is able to generate a kubeconfig for any registered clusters.
- A user is able to access a K8s API server with a generated kubeconfig.
- A user who is a root-org admin has a a super-user-role (i.e.,
  `cluster-admin`) in the K8s cluster. The user can access any namespaces
  in the K8s cluster.
- A user who is a sub-org admin can access only to a namespace owned
  by the org (from `kubectl` as well as workload page).

### Test Execution Steps

#### *Step 1*: Generate a kubeconfig for a cluster

In [ ]:
CLUSTER=<cluster-name>
cnatix clusters export-config --cluster ${CLUSTER}

#### *Step 2*: Verify that the current context is now set to the exported context.

In [ ]:
kubectl config get-contexts

#### *Step 3*: Run the `kubectl` command to verify that all namespaces are accessible.

In [ ]:
kubectl get pods --all-namespaces

#### *Step 4*: Generate a kubeconfig of another cluster and run the `kubectl` command to verify that all namespaces are accessible.

In [ ]:
ANOTHER_CLUSTER=<cluster-name>
cnatix clusters export-config --cluster ${ANOTHER_CLUSTER}

kubectl get pods --all-namespaces

#### *Step 5*: Login as a non-root-org admin and generate a kubeconfig

Login as a non-root-admin user. If you have already logged in., Visit
https://login.cloudnatix.com and click "Sign out" before running the
command.

In [ ]:
cnatix login

Verify that the role binding is set to the `<non-root-org>:admin`.

In [ ]:
cnatix iam users self

Then generate the kubeconfig.

In [ ]:
cnatix clusters export-config --cluster ${CLUSTER}

#### *Step 6*: Run the `kubectl` command to verify that only namespaces allocated to the org is accessible.

In [ ]:
# This command will get a permission error
kubectl get pods --all-namespaces

In [ ]:
# Run kubectl for a specific namespace. The command will succeed
kubectl get pods --namespace ${TEST_NAMESPACE}

#### *Step 7*: Visit Dashboard and verify that only namespaces allocated to the org is accessible.

Go to https://app.cloudnatix.com and login as as non-root-org admin user.

The "Workloads" should show only K8s workloads in the namespace.

## Test case 3: Identify a K8s workload and access workload logs/console with the `kubectl` command using the `cnatix` CLI across multiple clusters in minutes

### Expected Results

- A user is able to run `kubectl logs` and `kubectl exec` with a kubeconfig generated by the `cnatix` CLI.

### Test Execution Steps

#### *Step 1*: Generate a kubeconfig for a cluster

In [ ]:
cnatix clusters export-config --cluster ${CLUSTER}

#### *Step 2*: Find a target pod

Run `kubectl get pods --all-namespaces` and find a target pod. Please note that
only a root-org admin can view resources in all namespaces.

In [ ]:
kubectl get pods --all-namespaces

In [ ]:
TARGET_NAMESPACE=<target namespace name>
TARGET_POD=<target pod name>

#### *Step 3*: Run `kubectl log`

In [ ]:
kubectl logs --namespace ${TARGET_NAMESPACE} ${TARGET_POD}

#### *Step 4*: Run `kubectl exec`

If the pod you used for `kubectl log` is not suitable for this test, please choose some other pod.

In [ ]:
TARGET_NAMESPACE=<target namespace name>
TARGET_POD=<target pod name>

In [ ]:
# Assume /bin/sh is installed in the target pod.
kubectl exec -it --namespace ${TARGET_NAMESPACE} ${TARGET_POD} -- /bin/sh

Note that `kubectl exec` doesn't work on the Jupyter Lab code cell. Please try it
on your terminal.

## Test case 4: A federated view of K8s workloads across clusters, sort by Cost/Age, filter and identify workloads with availability issues or capacity optimization opportunities and the cluster they reside within

### Expected Results

- A user is able to view K8s workloads across multiple clusters in a single page.
- A user is able to see the current status of the workloads and where they're running.
- A user is able to sort workloads by cost or age.

### Test Execution Steps

Go to https://app.cloudnatix.com and click "Workloads". The page shows all K8s workloads across multiple clusters.

The "Status" column shows the number of pods grouped by their status
(Pending, Running, Succeeded, Failed, or Unknown). If there is a
workload that has a Pending or Failed pod, look at the "Cluster"
column to identify the cluster where the workload is running.

Click the sorting option on the "Cost" column or the "Age" column to sort workloads by the key.